In [ ]:
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import json
from glob import glob

In [ ]:
def evaluate(path,title):
    scores={}
    for i in glob(f'{path}/*/*_results.json'):
        with open(i,'r') as f:
            scores[os.path.basename(os.path.dirname(i))]=json.load(f)
    
    R2df=pd.DataFrame.from_dict(scores).loc[['R2_train','R2_test']].astype(float).T
    R2df['diff']=(R2df['R2_train']-R2df['R2_test']).abs()
    meltR2=R2df.reset_index().melt(id_vars='index',value_vars=['R2_train','R2_test']).sort_values(by='value')
    
    topR2=R2df[(R2df['R2_test']>0.75)&(R2df['R2_train']>0.75)].sort_values(['R2_train','R2_test'], ascending=[False,False])
    topmelt=topR2.reset_index().rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars='index',value_vars=['Train','Test']).sort_values(by='value')
    
    ax=sns.barplot(data=topmelt,x='index',y='value',hue='variable',palette=sns.color_palette('Paired',2))
    ax.tick_params(axis='x', labelrotation=90)
    ax.set_ylabel("R$^{2}$")
    ax.set_xlabel('ENCODER_ANSATZ')
    plt.title(title)
    plt.ylim(0,1)
    legend = plt.legend(loc=4, edgecolor="black")
    legend.get_frame().set_alpha(None)
    legend.get_frame().set_facecolor((1, 1, 1,1))
    plt.tight_layout()
    plt.savefig(f"{path}_metric.png",dpi=300,bbox_inches='tight')
    plt.show()
    
    
    with open(f'{path}_scores.json', 'w') as fp:
        json.dump(scores, fp)

In [ ]:
evaluate('./lin5qubits',"Linear Function + 5 qubits")

In [ ]:
evaluate('./sine5qubits',"Sine Function + 5 qubits")

In [ ]:
evaluate('./quad5qubits',"Quadratic Function + 5 qubits")